# Transfer Learning with Survival Models

We would like to see how we can use transfer learning to improve on survival model concordances. First, we build a global model that contains all patients across all diseases, while holding out 20% of patients for each disease type as the test set. Next, we would like to compare model performances for each individual disease type betweeen baseline and transfer models. Baseline models will be tuned for optimal number of hidden units, number of epochs, and learning rate. Transfer models will be trained on extracted features from the global model and tuned for optimal number of hidden units, number of epochs, and learning rate.

In [90]:
# Set backend for Keras to be Theano

from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

In [74]:
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop
import theano.tensor as T

from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from lifelines.datasets import load_rossi
from sklearn import preprocessing

from numpy.random import seed
seed(1)

In [3]:
# Custom loss function for survival model

def negative_log_likelihood(E):
	def loss(y_true,y_pred):
		hazard_ratio = T.exp(y_pred)
		log_risk = T.log(T.extra_ops.cumsum(hazard_ratio))
		uncensored_likelihood = y_pred.T - log_risk
		censored_likelihood = uncensored_likelihood * E
		neg_likelihood = -T.sum(censored_likelihood)
		return neg_likelihood
	return loss

In [89]:
# Sample of our data

import pandas as pd
import numpy as np
df = pd.read_csv('/home/nolelin/features_endpoints_2.csv')[['features', 'days_to_death', 'days_to_followup', 'case_id']]
df = df.dropna(subset=['days_to_death', 'days_to_followup'], how='all')
df.head(5)

,features,days_to_death,days_to_followup,case_id
0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,546.0,02f5ae33-a563-4ecb-9e33-dfa500a44931
1,"[0.4660520763729, 0.382840769064941, 0.4490283...",NaN,665.0,03ae8fdb-8a9c-401d-b8e3-b5ea57bad182
2,"[0.01407786535395935, 0.9705804498196411, 0.02...",NaN,2391.0,0ad2a2b3-c1bb-4597-b28e-a6385f93d09c
3,"[0.270973009254394, 0.0385691589193078, 0.9269...",NaN,417.0,0b31dbdf-0623-48e4-a0ec-017650dceda7
4,"[0.96800805140438, 0.0201589194544571, 0.95766...",NaN,106.0,0b6b1937-4024-4f2c-aeca-46387277755f


In [5]:
# Get disease codes for each disease type that we want to make comparisons between baseline/transfer

df2 = pd.read_csv('/home/nolelin/clinical3.csv')
df2 = df2.loc[df2['name'] == 'disease_code']
disease_codes = list(set(list(df2['value'])))
disease_codes

['ESCA',
 'DLBC',
 'READ',
 'GBM',
 'LGG',
 'PCPG',
 'STAD',
 'UCEC',
 'THCA',
 'CESC',
 'UCS',
 'THYM',
 'LIHC',
 'CHOL',
 'HNSC',
 'UVM',
 'SKCM',
 'COAD',
 'ACC',
 'LUAD',
 'TGCT',
 'LUSC',
 'MESO',
 'FPPP',
 'KIRC',
 'SARC',
 'KIRP',
 'BLCA',
 'LAML',
 'PRAD',
 'PAAD',
 'BRCA',
 'OV',
 'KICH']

In [6]:
# Get all case_ids of patients for each disease type

disease_caseids = {}
df2 = pd.read_csv('/home/nolelin/clinical3.csv')
for disease in disease_codes:
    diseasedf = df2.loc[df2['value'] == disease]
    disease_caseids[disease] = list(diseasedf['case_id'])

In [8]:
# Only train/validate on 80% of all patients for global model. So when doing transfer learning, we don't include test patients

from random import shuffle
df = df.dropna(subset=['days_to_death', 'days_to_followup'], how='all')
constrained_caseids = []
for disease in disease_caseids.keys():
    num_patients = len(disease_caseids[disease])
    shuffle(disease_caseids[disease])
    constrained_caseids.extend(disease_caseids[disease][0:int(0.8 * num_patients)])

In [9]:
# Split remaining 80% of all data into training and test (validation) for global model.
# We perform mean imputation and normalization on the methylation features
# For tuning, we keep track of concordances for each hyperparameter setting in global_errors dictionary

import numpy as np
import numpy.ma as ma
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

df = df.loc[df['case_id'].isin(constrained_caseids)]
df = df.sample(frac=1)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
x = np.asarray([[float(y) if y != 'nan' else np.nan for y in x[1:-1].split(", ")] for x in list(df['features'])])
x = np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x)
imp.fit(x)
x = imp.transform(x)
scaler = preprocessing.StandardScaler().fit(x)

train_size = int(0.75 * df.shape[0])
test_size = df.shape[0] - train_size
train = df.head(train_size)
test = df.tail(test_size)
#train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
output_cols = ['days_to_death', 'days_to_followup']
x_train = np.asarray([[float(y) if y != 'nan' else np.nan for y in x[1:-1].split(", ")] for x in list(train['features'])])
x_train = imp.transform(x_train)
x_train = scaler.transform(x_train)
days = np.asarray(train[output_cols])
y_train = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
e_train = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])

x_test = np.asarray([[float(y) if y != 'nan' else np.nan for y in x[1:-1].split(", ")] for x in list(test['features'])])
#x_test = np.where(np.isnan(x_test), ma.array(x_test, mask=np.isnan(x_test)).mean(axis=0), x_test)
x_test = imp.transform(x_test)
x_test = scaler.transform(x_test)
days = np.asarray(test[output_cols])
y_test = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
e_test = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])

sort_idx = np.argsort(y_train)[::-1]
x_train=x_train[sort_idx]
y_train=y_train[sort_idx]
e_train=e_train[sort_idx]

patience = 5
#disease_baseline_models = {}
global_errors = {}
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
units = [10, 50, 100, 200, 500, 1000, 2000]
for unit in units:
    for learning_rate in learning_rates:
        num_epochs = 0
        max_conc = 0
        num_patience = 0
        model = Sequential()
        model.add(Dense(unit, activation='relu', input_shape=(10000,)))
        model.add(BatchNormalization())
        model.add(Dense(1))
        adam = optimizers.Adam(lr=learning_rate)
        model.compile(loss=negative_log_likelihood(e_train), optimizer=adam)
        while True:
            model.fit(x_train, y_train, batch_size=x_train.shape[0], epochs=1, shuffle=False)
            hr_pred = model.predict(x_test)
            hr_pred=np.exp(hr_pred)
            ci=concordance_index(y_test,-hr_pred,e_test)
            if ci > max_conc:
                num_epochs += 1
                max_conc = ci
                num_patience = 0
            else:
                if num_patience < patience:
                    num_epochs += 1
                    num_patience += 1
                else:
                    break
        global_errors[(learning_rate, unit, num_epochs - patience)] = max_conc

Epoch 1/1
6535/6535 [==============================] - 2s 270us/step - loss: 16807.8651
Epoch 1/1
6535/6535 [==============================] - 2s 255us/step - loss: 17561.7955
Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 16365.0117
Epoch 1/1
6535/6535 [==============================] - 2s 251us/step - loss: 15930.2797
Epoch 1/1
6535/6535 [==============================] - 2s 247us/step - loss: 15887.9287
Epoch 1/1
6535/6535 [==============================] - 2s 233us/step - loss: 16008.5047
Epoch 1/1
6535/6535 [==============================] - 2s 244us/step - loss: 16112.8474
Epoch 1/1
6535/6535 [==============================] - 2s 250us/step - loss: 16114.9601
Epoch 1/1
6535/6535 [==============================] - 2s 250us/step - loss: 19040.6459
Epoch 1/1
6535/6535 [==============================] - 2s 241us/step - loss: 21064.2353
Epoch 1/1
6535/6535 [==============================] - 2s 233us/step - loss: 18852.1640
Epoch 1/1
6535/6535 [===========

6535/6535 [==============================] - 2s 236us/step - loss: 15130.4102
Epoch 1/1
6535/6535 [==============================] - 2s 233us/step - loss: 15109.5486
Epoch 1/1
6535/6535 [==============================] - 2s 238us/step - loss: 15088.1898
Epoch 1/1
6535/6535 [==============================] - 2s 244us/step - loss: 17160.3316
Epoch 1/1
6535/6535 [==============================] - 2s 235us/step - loss: 17304.7823
Epoch 1/1
6535/6535 [==============================] - 2s 236us/step - loss: 16201.8478
Epoch 1/1
6535/6535 [==============================] - 2s 235us/step - loss: 16023.6894
Epoch 1/1
6535/6535 [==============================] - 2s 242us/step - loss: 15883.8066
Epoch 1/1
6535/6535 [==============================] - 2s 245us/step - loss: 15780.2342
Epoch 1/1
6535/6535 [==============================] - 2s 246us/step - loss: 15706.4253
Epoch 1/1
6535/6535 [==============================] - 2s 244us/step - loss: 15654.7576
Epoch 1/1
6535/6535 [=====================

Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 15455.2558
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 15444.1750
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 15433.4799
Epoch 1/1
6535/6535 [==============================] - 2s 231us/step - loss: 15423.0270
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 15412.7514
Epoch 1/1
6535/6535 [==============================] - 2s 231us/step - loss: 15402.5875
Epoch 1/1
6535/6535 [==============================] - 2s 231us/step - loss: 15392.4784
Epoch 1/1
6535/6535 [==============================] - 2s 231us/step - loss: 15382.3987
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 15372.3805
Epoch 1/1
6535/6535 [==============================] - 2s 231us/step - loss: 15362.5191
Epoch 1/1
6535/6535 [==============================] - 2s 231us/step - loss: 15352.8083
Epoch 1/1
6535/6535 [===========

6535/6535 [==============================] - 2s 237us/step - loss: 16258.3119
Epoch 1/1
6535/6535 [==============================] - 2s 231us/step - loss: 16236.4891
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 16215.3833
Epoch 1/1
6535/6535 [==============================] - 2s 243us/step - loss: 16194.8902
Epoch 1/1
6535/6535 [==============================] - 2s 239us/step - loss: 16175.0994
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 16155.8963
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 16137.2830
Epoch 1/1
6535/6535 [==============================] - 2s 232us/step - loss: 16119.2333
Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 16101.7261
Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 16084.6855
Epoch 1/1
6535/6535 [==============================] - 2s 237us/step - loss: 16068.0900
Epoch 1/1
6535/6535 [=====================

Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 15461.3838
Epoch 1/1
6535/6535 [==============================] - 2s 238us/step - loss: 15458.0919
Epoch 1/1
6535/6535 [==============================] - 2s 240us/step - loss: 15454.8322
Epoch 1/1
6535/6535 [==============================] - 2s 236us/step - loss: 15451.5985
Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 15448.3893
Epoch 1/1
6535/6535 [==============================] - 2s 241us/step - loss: 15445.1983
Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 15442.0261
Epoch 1/1
6535/6535 [==============================] - 2s 238us/step - loss: 15438.8730
Epoch 1/1
6535/6535 [==============================] - 2s 234us/step - loss: 15435.7459
Epoch 1/1
6535/6535 [==============================] - 2s 243us/step - loss: 15432.6392
Epoch 1/1
6535/6535 [==============================] - 2s 242us/step - loss: 15429.5508
Epoch 1/1
6535/6535 [===========

...

Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 16522.7373
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 16455.6892
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 16366.4782
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 16263.7283
Epoch 1/1
6535/6535 [==============================] - 20s 3ms/step - loss: 16166.4338
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 16088.8920
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 16033.6756
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 15992.1759
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 15951.3581
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 15902.2301
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 15843.7688
Epoch 1/1
6535/6535 [======================

Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13706.5284
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13692.2050
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13677.9835
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13663.8534
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13649.8249
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13635.8939
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13622.0574
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13608.3155
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13594.6635
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13581.1009
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 13567.6262
Epoch 1/1
6535/6535 [======================

Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12667.0661
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12658.6908
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12650.3517
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12642.0503
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12633.7876
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12625.5662
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12617.3879
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12609.2418
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12601.1360
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12593.0724
Epoch 1/1
6535/6535 [==============================] - 19s 3ms/step - loss: 12585.0468
Epoch 1/1
6535/6535 [======================

Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14326.3781
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14292.2388
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14259.8138
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14227.8641
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14195.8345
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14164.1447
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14132.9863
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14102.1131
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14071.5956
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14041.7956
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 14012.8128
Epoch 1/1
6535/6535 [======================

Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12447.0690
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12435.0703
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12423.1677
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12411.3556
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12399.6330
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12388.0016
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12376.4574
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12364.9969
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12353.6233
Epoch 1/1
6535/6535 [==============================] - 37s 6ms/step - loss: 12342.3373


In [10]:
global_errors

{(1e-05, 10, 451): 0.721361615377613,
 (1e-05, 50, 470): 0.743185814420345,
 (1e-05, 100, 339): 0.7395780930341751,
 (1e-05, 200, 333): 0.7333687877002094,
 (1e-05, 500, 270): 0.7411316171503313,
 (1e-05, 1000, 220): 0.7433343394707717,
 (1e-05, 2000, 136): 0.7437332010981271,
 (0.0001, 10, 103): 0.7303803199517054,
 (0.0001, 50, 72): 0.7381922584910813,
 (0.0001, 100, 32): 0.7485566718889991,
 (0.0001, 200, 38): 0.7434828645211984,
 (0.0001, 500, 46): 0.7415520388656519,
 (0.0001, 1000, 78): 0.7489699069083313,
 (0.0001, 2000, 3): 0.6612383156301056,
 (0.001, 10, 53): 0.7578311030620116,
 (0.001, 50, 65): 0.748957929081684,
 (0.001, 100, 2): 0.5546045161197591,
 (0.001, 200, 2): 0.5674291751110344,
 (0.001, 500, 104): 0.7596768861483622,
 (0.001, 1000, 17): 0.6170497175628477,
 (0.001, 2000, 1): 0.6174324091242293,
 (0.01, 10, 82): 0.7278002960918747,
 (0.01, 50, 14): 0.6438824448181526,
 (0.01, 100, 2): 0.6477171460192891,
 (0.01, 200, 2): 0.6307960703146336,
 (0.01, 500, 7): 0.62833

In [11]:
# Get best learning rate, num. of units in our single hidden layers, and number of epochs to train for

max(global_errors, key=global_errors.get)

(0.001, 500, 104)

In [12]:
# Train global model according to best run
# Combine both training and test (validation) sets from above to use all possible data without adding in test set
total_x = np.vstack((x_train, x_test))
total_y = np.hstack((y_train, y_test))
total_e = np.hstack((e_train, e_test))
sort_idx = np.argsort(total_y)[::-1]
total_x=total_x[sort_idx]
total_y=total_y[sort_idx]
total_e=total_e[sort_idx]

adam = optimizers.Adam(lr=0.001)
model = Sequential()
model.add(Dense(500, activation='relu', input_shape=(10000,)))
model.add(BatchNormalization())
model.add(Dense(1))
model.compile(loss=negative_log_likelihood(total_e), optimizer=adam)
model.fit(total_x, total_y, batch_size=total_x.shape[0], epochs=104, shuffle=False)

Epoch 1/104
8714/8714 [==============================] - 13s 1ms/step - loss: 23778.2892
Epoch 2/104
8714/8714 [==============================] - 13s 2ms/step - loss: 115167.3714
Epoch 3/104
8714/8714 [==============================] - 13s 2ms/step - loss: 96497.4922
Epoch 4/104
8714/8714 [==============================] - 13s 1ms/step - loss: 75916.0164
Epoch 5/104
8714/8714 [==============================] - 14s 2ms/step - loss: 59693.5355
Epoch 6/104
8714/8714 [==============================] - 13s 2ms/step - loss: 49912.3753
Epoch 7/104
8714/8714 [==============================] - 13s 2ms/step - loss: 42111.9447
Epoch 8/104
8714/8714 [==============================] - 13s 1ms/step - loss: 36504.4542
Epoch 9/104
8714/8714 [==============================] - 13s 1ms/step - loss: 33258.0894
Epoch 10/104
8714/8714 [==============================] - 13s 1ms/step - loss: 31479.1308
Epoch 11/104
8714/8714 [==============================] - 13s 1ms/step - loss: 29863.3169
Epoch 12/104
8714/

8714/8714 [==============================] - 13s 1ms/step - loss: 20392.0806
Epoch 93/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20369.0311
Epoch 94/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20345.2860
Epoch 95/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20321.1448
Epoch 96/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20296.4729
Epoch 97/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20270.8473
Epoch 98/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20243.5299
Epoch 99/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20214.9865
Epoch 100/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20186.5391
Epoch 101/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20156.8111
Epoch 102/104
8714/8714 [==============================] - 13s 1ms/step - loss: 20125.6330
Epoch 103/104
8714/8

In [13]:
# Save our model to use later for transfer learning

model.save('/home/nolelin/all_patients_survival5.h5')

In [91]:
# Do same data pre-processing for each individual disease
# Don't include diseases that don't have at least 150 patients in training set and at least 20% of patients have died

import numpy as np
import numpy.ma as ma
from sklearn import preprocessing
from sklearn.impute import SimpleImputer

disease_x_train = {}
disease_x_test = {}
disease_x_val = {}
disease_y_train = {}
disease_y_test = {}
disease_y_val = {}
disease_e_train = {}
disease_e_test = {}
disease_e_val = {}
for disease in disease_codes:
    df = pd.read_csv('/home/nolelin/features_endpoints_2.csv')[['features', 'days_to_death', 'days_to_followup', 'case_id']]
    df = df.dropna(subset=['days_to_death', 'days_to_followup'], how='all')
    df = df.loc[df['case_id'].isin(disease_caseids[disease])]
    print disease, df.shape
    if disease == 'FPPP':
        continue
    #train_size = int(0.75 * df.shape[0])
    #test_size = df.shape[0] - train_size
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    x = np.asarray([[float(y) if y != 'nan' else np.nan for y in x[1:-1].split(", ")] for x in list(df['features'])])
    x = np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x)
    imp.fit(x)
    x = imp.transform(x)
    scaler = preprocessing.StandardScaler().fit(x)
    
    #train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    test = df.loc[~df['case_id'].isin(constrained_caseids)]
    df = df.loc[df['case_id'].isin(constrained_caseids)]
    df = df.sample(frac=1)
    train_size = int(0.75 * df.shape[0])
    test_size = df.shape[0] - train_size
    train = df.head(train_size)
    validate = df.tail(test_size)
    output_cols = ['days_to_death', 'days_to_followup']
    x_train = np.asarray([[float(y) if y != 'nan' else np.nan for y in x[1:-1].split(", ")] for x in list(train['features'])])
    x_train = imp.transform(x_train)
    x_train = scaler.transform(x_train)
    days = np.asarray(train[output_cols])
    y_train = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
    e_train = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])

    x_test = np.asarray([[float(y) if y != 'nan' else np.nan for y in x[1:-1].split(", ")] for x in list(test['features'])])
    #x_test = np.where(np.isnan(x_test), ma.array(x_test, mask=np.isnan(x_test)).mean(axis=0), x_test)
    x_test = imp.transform(x_test)
    x_test = scaler.transform(x_test)
    days = np.asarray(test[output_cols])
    y_test = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
    e_test = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
    
    x_val = np.asarray([[float(y) if y != 'nan' else np.nan for y in x[1:-1].split(", ")] for x in list(validate['features'])])
    #x_test = np.where(np.isnan(x_test), ma.array(x_test, mask=np.isnan(x_test)).mean(axis=0), x_test)
    x_val = imp.transform(x_val)
    x_val = scaler.transform(x_val)
    days = np.asarray(validate[output_cols])
    y_val = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
    e_val = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
    
    print x_train.shape, x_val.shape, x_test.shape
    
    if np.sum(e_train) < 0.2 * len(e_train) or np.sum(e_val) < 0.2 * len(e_val) or np.sum(e_test) < 0.2 * len(e_test) or len(e_train) < 150:
        print "skipping ", disease
        continue
    
    sort_idx = np.argsort(y_train)[::-1]
    x_train=x_train[sort_idx]
    y_train=y_train[sort_idx]
    e_train=e_train[sort_idx]
    
    disease_x_train[disease] = x_train
    disease_x_test[disease] = x_test
    disease_x_val[disease] = x_val
    disease_y_train[disease] = y_train
    disease_y_test[disease] = y_test
    disease_y_val[disease] = y_val
    disease_e_train[disease] = e_train
    disease_e_test[disease] = e_test
    disease_e_val[disease] = e_val

ESCA (185, 4)
(113, 10000) (38, 10000) (34, 10000)
skipping  ESCA
DLBC (48, 4)
(28, 10000) (10, 10000) (10, 10000)
skipping  DLBC
READ (164, 4)
(102, 10000) (34, 10000) (28, 10000)
skipping  READ
GBM (415, 4)
(253, 10000) (85, 10000) (77, 10000)
LGG (514, 4)
(307, 10000) (103, 10000) (104, 10000)
skipping  LGG
PCPG (179, 4)
(112, 10000) (38, 10000) (29, 10000)
skipping  PCPG
STAD (408, 4)
(246, 10000) (82, 10000) (80, 10000)
UCEC (545, 4)
(335, 10000) (112, 10000) (98, 10000)
skipping  UCEC
THCA (507, 4)
(310, 10000) (104, 10000) (93, 10000)
skipping  THCA
CESC (307, 4)
(185, 10000) (62, 10000) (60, 10000)
skipping  CESC
UCS (56, 4)
(34, 10000) (12, 10000) (10, 10000)
skipping  UCS
THYM (123, 4)
(73, 10000) (25, 10000) (25, 10000)
skipping  THYM
LIHC (376, 4)
(228, 10000) (77, 10000) (71, 10000)
CHOL (36, 4)
(23, 10000) (8, 10000) (5, 10000)
skipping  CHOL
HNSC (526, 4)
(321, 10000) (108, 10000) (97, 10000)
UVM (80, 4)
(48, 10000) (16, 10000) (16, 10000)
skipping  UVM
SKCM (460, 4)
(27

In [99]:
# Tuning for baseline models
# Find best hyperparameters for baseline model for each disease

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import regularizers

patience = 5
disease_baseline_errors = {}
units = [10, 50, 100, 200, 500, 1000, 2000]
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
for disease in disease_x_train.keys():
    for unit in units:
        for learning_rate in learning_rates:
            num_epochs = 0
            max_conc = 0
            num_patience = 0
            model = Sequential()
            model.add(Dense(unit, activation='relu', input_shape=(10000,)))
            model.add(BatchNormalization())
            model.add(Dense(1))
            adam = optimizers.Adam(lr=learning_rate)
            model.compile(loss=negative_log_likelihood(disease_e_train[disease]), optimizer=adam)
            while True:
                model.fit(disease_x_train[disease], disease_y_train[disease], batch_size=disease_x_train[disease].shape[0], epochs=1, shuffle=False)
                hr_pred = model.predict(disease_x_val[disease])
                hr_pred=np.exp(hr_pred)
                ci=concordance_index(disease_y_val[disease],-hr_pred,disease_e_val[disease])
                if ci > max_conc:
                    num_epochs += 1
                    max_conc = ci
                    num_patience = 0
                else:
                    if num_patience < patience:
                        num_epochs += 1
                        num_patience += 1
                    else:
                        break
            disease_baseline_errors[(disease, learning_rate, num_epochs - patience, unit)] = max_conc

Epoch 1/1
312/312 [==============================] - 0s 216us/step - loss: 627.9430
Epoch 1/1
312/312 [==============================] - 0s 180us/step - loss: 610.2291
Epoch 1/1
312/312 [==============================] - 0s 176us/step - loss: 559.6455
Epoch 1/1
312/312 [==============================] - 0s 175us/step - loss: 562.2664
Epoch 1/1
312/312 [==============================] - 0s 175us/step - loss: 569.2536
Epoch 1/1
312/312 [==============================] - 0s 184us/step - loss: 570.1264
Epoch 1/1
312/312 [==============================] - 0s 179us/step - loss: 567.0032
Epoch 1/1
312/312 [==============================] - 0s 184us/step - loss: 562.2639
Epoch 1/1
312/312 [==============================] - 0s 173us/step - loss: 557.0612
Epoch 1/1
312/312 [==============================] - 0s 181us/step - loss: 551.9568
Epoch 1/1
312/312 [==============================] - 0s 175us/step - loss: 547.3806
Epoch 1/1
312/312 [==============================] - 0s 175us/step - loss: 5

Epoch 1/1
312/312 [==============================] - 0s 230us/step - loss: 548.6667
Epoch 1/1
312/312 [==============================] - 0s 365us/step - loss: 541.1330
Epoch 1/1
312/312 [==============================] - 0s 314us/step - loss: 535.8985
Epoch 1/1
312/312 [==============================] - 0s 314us/step - loss: 533.0043
Epoch 1/1
312/312 [==============================] - 0s 314us/step - loss: 531.9109
Epoch 1/1
312/312 [==============================] - 0s 314us/step - loss: 531.3302
Epoch 1/1
312/312 [==============================] - 0s 314us/step - loss: 529.7753
Epoch 1/1
312/312 [==============================] - 0s 234us/step - loss: 526.8626
Epoch 1/1
312/312 [==============================] - 0s 231us/step - loss: 523.2059
Epoch 1/1
312/312 [==============================] - 0s 231us/step - loss: 519.6303
Epoch 1/1
312/312 [==============================] - 0s 229us/step - loss: 516.5083
Epoch 1/1
312/312 [==============================] - 0s 228us/step - loss: 5

Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 563.2719
Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 541.1000
Epoch 1/1
312/312 [==============================] - 0s 345us/step - loss: 526.2299
Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 712.9972
Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 625.8426
Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 574.8705
Epoch 1/1
312/312 [==============================] - 0s 346us/step - loss: 539.6661
Epoch 1/1
312/312 [==============================] - 0s 345us/step - loss: 512.7120
Epoch 1/1
312/312 [==============================] - 0s 346us/step - loss: 492.9985
Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 475.5324
Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 810.9091
Epoch 1/1
312/312 [==============================] - 0s 344us/step - loss: 7

Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 4106.5409
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 3101.9762
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 2146.6530
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 1265.1452
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 733.1700
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 641.2070
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 1101.0654
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 1020.3782
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 886.0949
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 897.7260
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 716.7642
Epoch 1/1
312/312 [==============================] - 0s 2ms/step - loss: 3287.9680
Epoch 1/1

Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 1015.0752
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 645.2797
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 901.5018
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 624.0841
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 564.5544
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 577.4652
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 533.2069
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 526.5412
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 692.0758
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 641.2252
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 610.3627
Epoch 1/1
312/312 [==============================] - 1s 3ms/step - loss: 587.1237
Epoch 1/1
312/3

...

Epoch 1/1
325/325 [==============================] - 0s 745us/step - loss: 884.7738
Epoch 1/1
325/325 [==============================] - 0s 747us/step - loss: 884.1816
Epoch 1/1
325/325 [==============================] - 0s 747us/step - loss: 883.5655
Epoch 1/1
325/325 [==============================] - 0s 745us/step - loss: 882.9347
Epoch 1/1
325/325 [==============================] - 0s 749us/step - loss: 882.2952
Epoch 1/1
325/325 [==============================] - 0s 748us/step - loss: 881.6590
Epoch 1/1
325/325 [==============================] - 0s 748us/step - loss: 881.0375
Epoch 1/1
325/325 [==============================] - 0s 748us/step - loss: 880.4408
Epoch 1/1
325/325 [==============================] - 0s 746us/step - loss: 879.8747
Epoch 1/1
325/325 [==============================] - 0s 750us/step - loss: 879.3333
Epoch 1/1
325/325 [==============================] - 0s 746us/step - loss: 878.7819
Epoch 1/1
325/325 [==============================] - 0s 747us/step - loss: 8

Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: 5118.5071
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: 4294.9190
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: 2388.7619
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: 1851.9708
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: 1392.9483
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: 1313.2759
Epoch 1/1
325/325 [==============================] - 1s 2ms/step - loss: 1184.7569
Epoch 1/1
325/325 [===============

325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 2436.7801
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 3ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 3ms/step - loss: nan
Epoch 1/1
325/325 [==============================] - 1s 3ms/step - loss: 2487.0482
Epoch 1/1
325/325 [==============================] - 1s 3ms/step - loss: 2

Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 828.5070
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 826.9345
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 825.2247
Epoch 1/1
325/325 [==============================] - 1s 3ms/step - loss: 823.6033
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 821.7409
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 820.1721
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 818.6201
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 816.7808
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 814.7951
Epoch 1/1
325/325 [==============================] - 1s 3ms/step - loss: 813.1315
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 811.2961
Epoch 1/1
325/325 [==============================] - 1s 4ms/step - loss: 809.6104
Epoch 1/1
325/32

Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 886.2931
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 883.4893
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 880.7872
Epoch 1/1
325/325 [==============================] - 2s 8ms/step - loss: 877.7771
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 2094.6161
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 1171.0903
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 1132.9397
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 1115.6171
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 1058.9333
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 984.8955
Epoch 1/1
325/325 [==============================] - 2s 8ms/step - loss: 933.4256
Epoch 1/1
325/325 [==============================] - 2s 7ms/step - loss: 915.6688
Epoch 1/1
3

In [100]:
disease_baseline_errors

{('BLCA', 1e-05, 1, 500): 0.536995006808897,
 ('BLCA', 1e-05, 5, 10): 0.5179300953245575,
 ('BLCA', 1e-05, 14, 1000): 0.607807535179301,
 ('BLCA', 1e-05, 15, 2000): 0.5265546981389015,
 ('BLCA', 1e-05, 26, 200): 0.5406264185201998,
 ('BLCA', 1e-05, 45, 100): 0.5710394916023604,
 ('BLCA', 1e-05, 61, 50): 0.5519745801180209,
 ('BLCA', 0.0001, 1, 500): 0.5406264185201998,
 ('BLCA', 0.0001, 7, 50): 0.6073536087153881,
 ('BLCA', 0.0001, 9, 2000): 0.5805719473445302,
 ('BLCA', 0.0001, 16, 10): 0.6241488878801634,
 ('BLCA', 0.0001, 18, 200): 0.6082614616432138,
 ('BLCA', 0.0001, 20, 100): 0.5351793009532456,
 ('BLCA', 0.0001, 32, 1000): 0.6305038583749433,
 ('BLCA', 0.001, 2, 200): 0.610531093962778,
 ('BLCA', 0.001, 4, 2000): 0.6200635497049478,
 ('BLCA', 0.001, 12, 1000): 0.5551520653654108,
 ('BLCA', 0.001, 19, 50): 0.6817975487970949,
 ('BLCA', 0.001, 20, 10): 0.6268724466636405,
 ('BLCA', 0.001, 20, 500): 0.6572855197458012,
 ('BLCA', 0.001, 33, 100): 0.666817975487971,
 ('BLCA', 0.01, 1

In [101]:
# Train each baseline for each disease based on best hyperparameter settings

disease_baseline_models = {}
for disease in disease_x_train.keys():
    this_disease_runs = [x for x in disease_baseline_errors.keys() if disease in x[0]]
    highest_conc = 0
    best_run = None
    for run in this_disease_runs:
        if disease_baseline_errors[run] > highest_conc:
            highest_conc = disease_baseline_errors[run]
            best_run = run
    print best_run, disease_baseline_errors[best_run]
    total_x = np.vstack((disease_x_train[disease], disease_x_val[disease]))
    total_y = np.hstack((disease_y_train[disease], disease_y_val[disease]))
    total_e = np.hstack((disease_e_train[disease], disease_e_val[disease]))
    #total_x = disease_x_train[disease]
    #total_y = disease_y_train[disease]
    #total_e = disease_e_train[disease]
    sort_idx = np.argsort(total_y)[::-1]
    total_x=total_x[sort_idx]
    total_y=total_y[sort_idx]
    total_e=total_e[sort_idx]
    model = Sequential()
    model.add(Dense(best_run[3], activation='relu', input_shape=(10000,)))
    model.add(BatchNormalization())
    model.add(Dense(1))
    adam = optimizers.Adam(lr=best_run[1])
    model.compile(loss=negative_log_likelihood(total_e), optimizer=adam)
    model.fit(total_x, total_y, batch_size=total_x.shape[0], epochs=best_run[2], shuffle=False)
    disease_baseline_models[disease] = model

('LUAD', 0.01, 17, 1000) 0.6310160427807486
Epoch 1/17
416/416 [==============================] - 1s 3ms/step - loss: 883.9335
Epoch 2/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 3/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 4/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 5/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 6/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 7/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 8/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 9/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 10/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 11/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 12/17
416/416 [==============================] - 1s 3ms/step - loss: nan
Epoch 13/17


Epoch 59/63
338/338 [==============================] - 0s 645us/step - loss: 933.2575
Epoch 60/63
338/338 [==============================] - 0s 643us/step - loss: 929.2988
Epoch 61/63
338/338 [==============================] - 0s 645us/step - loss: 925.1697
Epoch 62/63
338/338 [==============================] - 0s 643us/step - loss: 920.9981
Epoch 63/63
338/338 [==============================] - 0s 642us/step - loss: 916.6438
('BLCA', 0.001, 19, 50) 0.6817975487970949
Epoch 1/19
352/352 [==============================] - 0s 262us/step - loss: 939.5586
Epoch 2/19
352/352 [==============================] - 0s 244us/step - loss: 1274.7777
Epoch 3/19
352/352 [==============================] - 0s 229us/step - loss: 933.9278
Epoch 4/19
352/352 [==============================] - 0s 228us/step - loss: 872.0976
Epoch 5/19
352/352 [==============================] - 0s 228us/step - loss: 810.4253
Epoch 6/19
352/352 [==============================] - 0s 231us/step - loss: 769.6260
Epoch 7/19
352/3

In [102]:
# Now get test concordances for each disease

disease_baseline_cis = {}
for disease in disease_x_train.keys():
    hr_pred=disease_baseline_models[disease].predict(disease_x_test[disease])
    hr_pred=np.exp(hr_pred)
    ci=concordance_index(disease_y_test[disease],-hr_pred,disease_e_test[disease])
    disease_baseline_cis[disease] = ci

In [103]:
disease_baseline_cis

{'BLCA': 0.6662531017369727,
 'COAD': 0.40787716955941256,
 'GBM': 0.5724937862468932,
 'HNSC': 0.5178786661309763,
 'KIRC': 0.6847940865892291,
 'LIHC': 0.3761854583772392,
 'LUAD': 0.5,
 'LUSC': 0.5459231490159325,
 'OV': 0.5,
 'SARC': 0.43562610229276894,
 'SKCM': 0.6168582375478927,
 'STAD': 0.650227617602428}

In [104]:
np.mean(disease_baseline_cis.values())

0.5395097812583122

In [22]:
# Load in the global model we previously saved

from keras.models import load_model
import keras.losses
full_model = load_model('/home/nolelin/all_patients_survival5.h5', custom_objects={'loss': negative_log_likelihood(total_e)})
full_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_71 (Dense)             (None, 500)               5000500   
_________________________________________________________________
batch_normalization_36 (Batc (None, 500)               2000      
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 501       
Total params: 5,003,001
Trainable params: 5,002,001
Non-trainable params: 1,000
_________________________________________________________________


In [24]:
# We want to use the global model as a feature extractor, something to transform our smaller disease-specific datasets
# To do this, we remove the last layer of the global model, which has output shape of 500, 
    # and ask the model to predict (transform) our original datasets of shape 10000 to a shape of 500
# Then we simply train a new model with the transformed input features and compare it to the baseline model for each disease

from keras.models import Model
from keras.layers import Input, Dense
model2= Model(inputs=full_model.input, outputs=full_model.get_layer('batch_normalization_36').output)
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_71_input (InputLayer)  (None, 10000)             0         
_________________________________________________________________
dense_71 (Dense)             (None, 500)               5000500   
_________________________________________________________________
batch_normalization_36 (Batc (None, 500)               2000      
Total params: 5,002,500
Trainable params: 5,001,500
Non-trainable params: 1,000
_________________________________________________________________


In [92]:
# Tuning for transfer models
# We perform normalization after transforming our datasets with the global model and tune a new model

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import regularizers

patience = 5
disease_transfer_errors = {}
disease_scalers = {}
units = [10, 50, 100, 200, 500, 1000, 2000]
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
for disease in disease_x_train.keys():
    new_x = model2.predict(np.vstack((disease_x_train[disease], disease_x_val[disease], disease_x_test[disease])))
    new_scaler = preprocessing.StandardScaler().fit(new_x)
    new_features = model2.predict(disease_x_train[disease])
    new_features = new_scaler.transform(new_features)
    new_validation = model2.predict(disease_x_val[disease])
    new_validation = new_scaler.transform(new_validation)
    disease_scalers[disease] = new_scaler
    for unit in units:
        for learning_rate in learning_rates:
            num_epochs = 0
            max_conc = 0
            num_patience = 0
            model = Sequential()
            model.add(Dense(unit, activation='linear', input_shape=(new_features.shape[1],)))
            model.add(BatchNormalization())
            model.add(Dense(1))
            adam = optimizers.Adam(lr=learning_rate)
            model.compile(loss=negative_log_likelihood(disease_e_train[disease]), optimizer=adam)
            while True:
                model.fit(new_features, disease_y_train[disease], batch_size=new_features.shape[0], epochs=1, shuffle=False)
                hr_pred = model.predict(new_validation)
                hr_pred=np.exp(hr_pred)
                ci=concordance_index(disease_y_val[disease],-hr_pred,disease_e_val[disease])
                if ci > max_conc:
                    num_epochs += 1
                    max_conc = ci
                    num_patience = 0
                else:
                    if num_patience < patience:
                        num_epochs += 1
                        num_patience += 1
                    else:
                        break
            disease_transfer_errors[(disease, learning_rate, num_epochs - patience, unit)] = max_conc

Epoch 1/1
312/312 [==============================] - 0s 12us/step - loss: 614.8657
Epoch 1/1
312/312 [==============================] - 0s 11us/step - loss: 1062.3576
Epoch 1/1
312/312 [==============================] - 0s 10us/step - loss: 768.8464
Epoch 1/1
312/312 [==============================] - 0s 10us/step - loss: 599.1079
Epoch 1/1
312/312 [==============================] - 0s 10us/step - loss: 577.6225
Epoch 1/1
312/312 [==============================] - 0s 11us/step - loss: 597.8863
Epoch 1/1
312/312 [==============================] - 0s 10us/step - loss: 618.5267
Epoch 1/1
312/312 [==============================] - 0s 11us/step - loss: 632.1651
Epoch 1/1
312/312 [==============================] - 0s 11us/step - loss: 638.4636
Epoch 1/1
312/312 [==============================] - 0s 10us/step - loss: 638.6932
Epoch 1/1
312/312 [==============================] - 0s 10us/step - loss: 634.4502
Epoch 1/1
312/312 [==============================] - 0s 12us/step - loss: 721.5150
Epo

Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 606.9129
Epoch 1/1
312/312 [==============================] - 0s 15us/step - loss: 597.6673
Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 580.1327
Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 562.0605
Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 551.8333
Epoch 1/1
312/312 [==============================] - 0s 15us/step - loss: 547.0508
Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 545.9974
Epoch 1/1
312/312 [==============================] - 0s 15us/step - loss: 546.3701
Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 543.7625
Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 537.5306
Epoch 1/1
312/312 [==============================] - 0s 14us/step - loss: 531.2809
Epoch 1/1
312/312 [==============================] - 0s 16us/step - loss: 753.4952
Epoc

Epoch 1/1
312/312 [==============================] - 0s 39us/step - loss: 716.7726
Epoch 1/1
312/312 [==============================] - 0s 40us/step - loss: 702.7474
Epoch 1/1
312/312 [==============================] - 0s 40us/step - loss: 680.4757
Epoch 1/1
312/312 [==============================] - 0s 42us/step - loss: 677.9662
Epoch 1/1
312/312 [==============================] - 0s 42us/step - loss: 734.7320
Epoch 1/1
312/312 [==============================] - 0s 41us/step - loss: 654.2381
Epoch 1/1
312/312 [==============================] - 0s 39us/step - loss: 590.6050
Epoch 1/1
312/312 [==============================] - 0s 39us/step - loss: 568.4286
Epoch 1/1
312/312 [==============================] - 0s 39us/step - loss: 556.5471
Epoch 1/1
312/312 [==============================] - 0s 39us/step - loss: 547.3239
Epoch 1/1
312/312 [==============================] - 0s 41us/step - loss: 541.4735
Epoch 1/1
312/312 [==============================] - 0s 40us/step - loss: 537.3614
Epoc

Epoch 1/1
312/312 [==============================] - 0s 94us/step - loss: 576.9190
Epoch 1/1
312/312 [==============================] - 0s 93us/step - loss: 561.7225
Epoch 1/1
312/312 [==============================] - 0s 93us/step - loss: 557.0063
Epoch 1/1
312/312 [==============================] - 0s 93us/step - loss: 558.6026
Epoch 1/1
312/312 [==============================] - 0s 93us/step - loss: 554.6761
Epoch 1/1
312/312 [==============================] - 0s 92us/step - loss: 550.2473
Epoch 1/1
312/312 [==============================] - 0s 97us/step - loss: 730.3280
Epoch 1/1
312/312 [==============================] - 0s 96us/step - loss: 713.8798
Epoch 1/1
312/312 [==============================] - 0s 93us/step - loss: 699.0194
Epoch 1/1
312/312 [==============================] - 0s 93us/step - loss: 685.7787
Epoch 1/1
312/312 [==============================] - 0s 92us/step - loss: 674.1478
Epoch 1/1
312/312 [==============================] - 0s 93us/step - loss: 664.0739
Epoc

Epoch 1/1
312/312 [==============================] - 0s 364us/step - loss: 630.6082
Epoch 1/1
312/312 [==============================] - 0s 364us/step - loss: 624.4572
Epoch 1/1
312/312 [==============================] - 0s 382us/step - loss: 619.3200
Epoch 1/1
312/312 [==============================] - 0s 399us/step - loss: 614.9320
Epoch 1/1
312/312 [==============================] - 0s 365us/step - loss: 611.0727
Epoch 1/1
312/312 [==============================] - 0s 365us/step - loss: 607.5591
Epoch 1/1
312/312 [==============================] - 0s 365us/step - loss: 604.2450
Epoch 1/1
312/312 [==============================] - 0s 365us/step - loss: 601.0230
Epoch 1/1
312/312 [==============================] - 0s 378us/step - loss: 597.8234
Epoch 1/1
312/312 [==============================] - 0s 371us/step - loss: 594.6095
Epoch 1/1
312/312 [==============================] - 0s 364us/step - loss: 591.3697
Epoch 1/1
312/312 [==============================] - 0s 365us/step - loss: 5

...

Epoch 1/1
267/267 [==============================] - 0s 413us/step - loss: 290.7508
Epoch 1/1
267/267 [==============================] - 0s 432us/step - loss: 303.1038
Epoch 1/1
267/267 [==============================] - 0s 453us/step - loss: 300.2512
Epoch 1/1
267/267 [==============================] - 0s 415us/step - loss: 297.6765
Epoch 1/1
267/267 [==============================] - 0s 425us/step - loss: 295.3232
Epoch 1/1
267/267 [==============================] - 0s 426us/step - loss: 293.1834
Epoch 1/1
267/267 [==============================] - 0s 424us/step - loss: 291.2462
Epoch 1/1
267/267 [==============================] - 0s 414us/step - loss: 289.4945
Epoch 1/1
267/267 [==============================] - 0s 414us/step - loss: 287.9114
Epoch 1/1
267/267 [==============================] - 0s 414us/step - loss: 286.4796
Epoch 1/1
267/267 [==============================] - 0s 433us/step - loss: 285.1786
Epoch 1/1
267/267 [==============================] - 0s 429us/step - loss: 2

Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 986.7388
Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 982.0361
Epoch 1/1
325/325 [==============================] - 0s 15us/step - loss: 977.6483
Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 973.5035
Epoch 1/1
325/325 [==============================] - 0s 15us/step - loss: 969.5722
Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 965.9191
Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 962.7619
Epoch 1/1
325/325 [==============================] - 0s 17us/step - loss: 960.4993
Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 959.4674
Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 959.1889
Epoch 1/1
325/325 [==============================] - 0s 17us/step - loss: 958.2208
Epoch 1/1
325/325 [==============================] - 0s 16us/step - loss: 955.7755
Epoc

Epoch 1/1
325/325 [==============================] - 0s 44us/step - loss: nan
Epoch 1/1
325/325 [==============================] - 0s 46us/step - loss: nan
Epoch 1/1
325/325 [==============================] - 0s 49us/step - loss: 954.3352
Epoch 1/1
325/325 [==============================] - 0s 48us/step - loss: 4630.2493
Epoch 1/1
325/325 [==============================] - 0s 47us/step - loss: 1134.0463
Epoch 1/1
325/325 [==============================] - 0s 47us/step - loss: 1062.1652
Epoch 1/1
325/325 [==============================] - 0s 47us/step - loss: 1076.8607
Epoch 1/1
325/325 [==============================] - 0s 48us/step - loss: 1083.2520
Epoch 1/1
325/325 [==============================] - 0s 49us/step - loss: 1085.8828
Epoch 1/1
325/325 [==============================] - 0s 46us/step - loss: 1239.0235
Epoch 1/1
325/325 [==============================] - 0s 44us/step - loss: 971.5493
Epoch 1/1
325/325 [==============================] - 0s 45us/step - loss: 982.9769
Epoch 1

Epoch 1/1
325/325 [==============================] - 0s 106us/step - loss: 914.3220
Epoch 1/1
325/325 [==============================] - 0s 105us/step - loss: 918.1539
Epoch 1/1
325/325 [==============================] - 0s 106us/step - loss: 922.0777
Epoch 1/1
325/325 [==============================] - 0s 105us/step - loss: 924.6121
Epoch 1/1
325/325 [==============================] - 0s 106us/step - loss: 925.9570
Epoch 1/1
325/325 [==============================] - 0s 111us/step - loss: 1473.9450
Epoch 1/1
325/325 [==============================] - 0s 113us/step - loss: 1433.9287
Epoch 1/1
325/325 [==============================] - 0s 111us/step - loss: 1394.2921
Epoch 1/1
325/325 [==============================] - 0s 108us/step - loss: 1355.1388
Epoch 1/1
325/325 [==============================] - 0s 109us/step - loss: 1316.6283
Epoch 1/1
325/325 [==============================] - 0s 109us/step - loss: 1278.9989
Epoch 1/1
325/325 [==============================] - 0s 108us/step - l

Epoch 1/1
325/325 [==============================] - 0s 419us/step - loss: 1585.1914
Epoch 1/1
325/325 [==============================] - 0s 439us/step - loss: 1810.2374
Epoch 1/1
325/325 [==============================] - 0s 420us/step - loss: 966.6308
Epoch 1/1
325/325 [==============================] - 0s 427us/step - loss: 959.0441
Epoch 1/1
325/325 [==============================] - 0s 419us/step - loss: 971.7097
Epoch 1/1
325/325 [==============================] - 0s 668us/step - loss: 980.0518
Epoch 1/1
325/325 [==============================] - 0s 542us/step - loss: 985.4218
Epoch 1/1
325/325 [==============================] - 0s 419us/step - loss: 992.1662
Epoch 1/1
325/325 [==============================] - 0s 427us/step - loss: 1093.5895
Epoch 1/1
325/325 [==============================] - 0s 420us/step - loss: 1018.6866
Epoch 1/1
325/325 [==============================] - 0s 419us/step - loss: 962.5199
Epoch 1/1
325/325 [==============================] - 0s 420us/step - los

In [93]:
disease_transfer_errors

{('BLCA', 1e-05, 1, 10): 0.45982750794371313,
 ('BLCA', 1e-05, 1, 50): 0.5901044030867,
 ('BLCA', 1e-05, 1, 1000): 0.5020426690876079,
 ('BLCA', 1e-05, 5, 100): 0.45982750794371313,
 ('BLCA', 1e-05, 48, 2000): 0.651838402178847,
 ('BLCA', 1e-05, 197, 500): 0.6768043576940536,
 ('BLCA', 1e-05, 299, 200): 0.7208352246935996,
 ('BLCA', 0.0001, 1, 10): 0.48252383113935543,
 ('BLCA', 0.0001, 1, 50): 0.4820699046754426,
 ('BLCA', 0.0001, 1, 200): 0.4770767135724013,
 ('BLCA', 0.0001, 7, 100): 0.5469813890149796,
 ('BLCA', 0.0001, 14, 2000): 0.7199273717657739,
 ('BLCA', 0.0001, 18, 1000): 0.7280980481162052,
 ('BLCA', 0.0001, 22, 500): 0.7199273717657739,
 ('BLCA', 0.001, 7, 200): 0.6055379028597367,
 ('BLCA', 0.001, 14, 500): 0.6472991375397186,
 ('BLCA', 0.001, 15, 1000): 0.672719019518838,
 ('BLCA', 0.001, 18, 2000): 0.6345891965501589,
 ('BLCA', 0.001, 20, 100): 0.7244666364049024,
 ('BLCA', 0.001, 32, 50): 0.7280980481162052,
 ('BLCA', 0.001, 60, 10): 0.7267362687244666,
 ('BLCA', 0.01,

In [94]:
# Train our transfer learned models

disease_transfer_models = {}
for disease in disease_x_train.keys():
    this_disease_runs = [x for x in disease_transfer_errors.keys() if disease in x[0]]
    highest_conc = 0
    best_run = None
    for run in this_disease_runs:
        if disease_transfer_errors[run] > highest_conc:
            highest_conc = disease_transfer_errors[run]
            best_run = run
    print best_run, disease_transfer_errors[best_run]
    total_x = np.vstack((disease_x_train[disease], disease_x_val[disease]))
    total_x = model2.predict(total_x)
    total_x = disease_scalers[disease].transform(total_x)
    total_y = np.hstack((disease_y_train[disease], disease_y_val[disease]))
    total_e = np.hstack((disease_e_train[disease], disease_e_val[disease]))
    #total_x = disease_x_train[disease]
    #total_y = disease_y_train[disease]
    #total_e = disease_e_train[disease]
    sort_idx = np.argsort(total_y)[::-1]
    total_x=total_x[sort_idx]
    total_y=total_y[sort_idx]
    total_e=total_e[sort_idx]
    model = Sequential()
    model.add(Dense(best_run[3], activation='linear', input_shape=(total_x.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dense(1))
    adam = optimizers.Adam(lr=best_run[1])
    model.compile(loss=negative_log_likelihood(total_e), optimizer=adam)
    model.fit(total_x, total_y, batch_size=total_x.shape[0], epochs=best_run[2], shuffle=False)
    disease_transfer_models[disease] = model

('LUAD', 0.001, 9, 10) 0.6631016042780749
Epoch 1/9
416/416 [==============================] - 0s 11us/step - loss: 926.7114
Epoch 2/9
416/416 [==============================] - 0s 11us/step - loss: 874.3834
Epoch 3/9
416/416 [==============================] - 0s 10us/step - loss: 828.8613
Epoch 4/9
416/416 [==============================] - 0s 10us/step - loss: 789.8319
Epoch 5/9
416/416 [==============================] - 0s 10us/step - loss: 766.7281
Epoch 6/9
416/416 [==============================] - 0s 10us/step - loss: 768.6754
Epoch 7/9
416/416 [==============================] - 0s 10us/step - loss: 770.5940
Epoch 8/9
416/416 [==============================] - 0s 10us/step - loss: 765.5974
Epoch 9/9
416/416 [==============================] - 0s 10us/step - loss: 761.3709
('LUSC', 0.001, 2, 100) 0.6890215893431327
Epoch 1/2
403/403 [==============================] - 0s 24us/step - loss: 945.6975
Epoch 2/2
403/403 [==============================] - 0s 23us/step - loss: 947.9283
('

Epoch 75/98
305/305 [==============================] - 0s 32us/step - loss: 481.4529
Epoch 76/98
305/305 [==============================] - 0s 32us/step - loss: 480.9767
Epoch 77/98
305/305 [==============================] - 0s 32us/step - loss: 480.5082
Epoch 78/98
305/305 [==============================] - 0s 32us/step - loss: 480.0446
Epoch 79/98
305/305 [==============================] - 0s 32us/step - loss: 479.5851
Epoch 80/98
305/305 [==============================] - 0s 33us/step - loss: 479.1306
Epoch 81/98
305/305 [==============================] - 0s 33us/step - loss: 478.6821
Epoch 82/98
305/305 [==============================] - 0s 32us/step - loss: 478.2387
Epoch 83/98
305/305 [==============================] - 0s 32us/step - loss: 477.7988
Epoch 84/98
305/305 [==============================] - 0s 33us/step - loss: 477.3623
Epoch 85/98
305/305 [==============================] - 0s 33us/step - loss: 476.9302
Epoch 86/98
305/305 [==============================] - 0s 33us/st

Epoch 18/32
352/352 [==============================] - 0s 15us/step - loss: 737.0596
Epoch 19/32
352/352 [==============================] - 0s 14us/step - loss: 734.5036
Epoch 20/32
352/352 [==============================] - 0s 14us/step - loss: 732.0479
Epoch 21/32
352/352 [==============================] - 0s 15us/step - loss: 729.2122
Epoch 22/32
352/352 [==============================] - 0s 14us/step - loss: 726.5842
Epoch 23/32
352/352 [==============================] - 0s 15us/step - loss: 724.9065
Epoch 24/32
352/352 [==============================] - 0s 15us/step - loss: 723.2683
Epoch 25/32
352/352 [==============================] - 0s 14us/step - loss: 720.5764
Epoch 26/32
352/352 [==============================] - 0s 14us/step - loss: 717.7196
Epoch 27/32
352/352 [==============================] - 0s 14us/step - loss: 715.7329
Epoch 28/32
352/352 [==============================] - 0s 14us/step - loss: 714.1481
Epoch 29/32
352/352 [==============================] - 0s 14us/st

In [95]:
# Test our transfer learned models

disease_transfer_cis = {}
for disease in disease_x_train.keys():
    new_test = model2.predict(disease_x_test[disease])
    new_test = disease_scalers[disease].transform(new_test)
    hr_pred=disease_transfer_models[disease].predict(new_test)
    hr_pred=np.exp(hr_pred)
    ci=concordance_index(disease_y_test[disease],-hr_pred,disease_e_test[disease])
    disease_transfer_cis[disease] = ci

In [96]:
disease_transfer_cis

{'BLCA': 0.7977667493796526,
 'COAD': 0.5193591455273698,
 'GBM': 0.5265120132560066,
 'HNSC': 0.7717959019686621,
 'KIRC': 0.6182682154171066,
 'LIHC': 0.7913593256059009,
 'LUAD': 0.5403321470937129,
 'LUSC': 0.6326148078725399,
 'OV': 0.477533960292581,
 'SARC': 0.7019400352733686,
 'SKCM': 0.8386015325670498,
 'STAD': 0.5811836115326252}

Literature suggests that we train a linear estimator for the transformed/transfered features, but we still checked how relu activation compares to linear activation. We also looked at if normalization helped test results. Based on different settings, we see that linear activation with normalization had best test performance.

In [33]:
# Linear activation, no normalization
np.mean(disease_transfer_cis.values())

0.6424976270131417

In [40]:
# Relu activation, no normalization
np.mean(disease_transfer_cis.values())

0.6183900727649989

In [97]:
# Linear activation, with normalization
np.mean(disease_transfer_cis.values())

0.6497722871488812

In [52]:
# Relu activation, with normalization
np.mean(disease_transfer_cis.values())

0.6211637006966484

Below is a comparison of baseline vs. transfer models for different diseases. It should be noted that there are slight differences in results for different runs and seeding should be used for consistent results. But in general, we do see that transfer learned models perform better on average than baseline models.

In [105]:
import plotly.plotly as py
import plotly.graph_objs as go

baseline = []
transfer = []

for disease in disease_x_train.keys():
    baseline.append(disease_baseline_cis[disease])
    transfer.append(disease_transfer_cis[disease])

baseline_concordance = go.Bar(
    x=disease_x_train.keys(),
    y=baseline,
    text=baseline,
    textposition = 'auto',
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6,
    name='baseline'
)

transfer_concordance = go.Bar(
    x=disease_x_train.keys(),
    y=transfer,
    text=transfer,
    textposition = 'auto',
    marker=dict(
        color='rgb(58,200,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6,
    name='transfer'
)

data = [baseline_concordance,transfer_concordance]

py.iplot(data, filename='grouped-bar-direct-labels')